# Interactive Car-Parrinello simulation on Quantum ESPRESSO

In order to assess the *Jupyter-workflow* capabilities to enable interactive simulations of realistic, large-scale systems, we implement a Notebook describing a multi-step simulation workflow in Quantum ESPRESSO.

In particular, the analyzed workflow implements a Car-Parrinello simulation of 32 Water Molecules, with the aim to sample the water at different temperatures using a Nose-hover thermostat together with a reference microcanonical trajectory.

The workflow is composed of six steps, which were executed on top of the PBS-managed *davinci-1* facility, the HPC centre of the Leonard S.p.A. company.

This time we did not use a Singularity container, but ran the steps directly on bare metal to fully exploit compile-time optimisations of a Quantum ESPRESSO executable compiled with Intel OneAPI. Nevertheless, we packed both the executable (named `cp-oneapi.x`) and all the input files inside the `INPDIR` directory.

In order to replicate the experiment in a fully-optimised execution environment, you need to compile the Quantum ESPRESSO `cp` executable directly on your facility. The [node-details.txt file](https://raw.githubusercontent.com/alpha-unito/jupyter-workflow/master/examples/quantum-espresso/node-details.txt) provides some information on the libraries used to compile Quantum ESPRESSO. Then, you need to set the correct credentials in the Notebook metadata (e.g. `username` and `sshKey`) and, if necessary, modify the PBS script template at `environment/pbs_espresso`.

Moreover, please note that both the input directory and the output directory should preserve their names (i.e., `INPDIR` and `OUTDIR`, respectively) because their paths are directly encoded inside Quantum ESPRESSO input files. Data dependencies are propagated between subsequent steps through the `OUTDIR` directory

The first step performs an electronic minimization, in order to bring the electronic density in the ground state.

In [2]:
import time
stime = time.time()

In [3]:
%%bash -s "$input_dir"
cp -r $1 .
mkdir OUTDIR
mpirun -np 16 -f $PBS_NODEFILE sh -c "./INPDIR/cp-oneapi.x < ./INPDIR/h2o.in.00 > ./OUTDIR/h2o.out.00"

In [4]:
print(time.time() - stime)
stime = time.time()

45.341166734695435


The second step performs a randomization of the ionic degree of freedom, followed by a new electronic minimisation

In [5]:
%%bash -s "$input_dir" "$output_dir"
cp -r $1 $2 .
mpirun -np 16 -f $PBS_NODEFILE sh -c "./INPDIR/cp-oneapi.x < ./INPDIR/h2o.in.01 > ./OUTDIR/h2o.out.01"

In [6]:
print(time.time() - stime)
stime = time.time()

16.089718103408813


The third step performs one thousand steps of a combined ionic and electronic microcanonical simulation to integrate the Car-Parrinello lagrangian

In [7]:
%%bash -s "$input_dir" "$output_dir"
cp -r $1 $2 .
mpirun -np 16 -f $PBS_NODEFILE sh -c "./INPDIR/cp-oneapi.x < ./INPDIR/h2o.in.02 > ./OUTDIR/h2o.out.0"

In [8]:
print(time.time() - stime)
stime = time.time()

174.44248843193054


At this point, the workflow forks in three different directions, which can all be executed in parallel. In particular, they perform:

1. A continuation of the microcanonical simulation of the third step, used as a reference
2. A continuation of the third step using a thermostat set at 350 Kelvin degrees
3. A continuation of the third step using a thermostat set at 400 Kelvin degrees

This configuration can be easily represented in *Jupyter-workflow* as a `scatter` operation on the input configuration files. The three resulting jobs are then concurrently scheduled on the remote HPC facility, while the *Jupyter-workflow* driver waits for their termination.

In [9]:
input_list = ['b0', 'b1', 'b2']

In [11]:
%%bash -s "$input_list" "$input_dir" "$output_dir"
INPUTS=($(echo $1 | tr -d '[],'))
cp -r $2 $3 .
for INPUTVAR in ${INPUTS[*]}; do
  INPUT=$(echo $INPUTVAR | tr -d \')
  mkdir OUTDIR.$INPUT
  cp -r -P OUTDIR/*  OUTDIR.$INPUT
  mpirun -np 16 -f $PBS_NODEFILE sh -c "./INPDIR/cp-oneapi.x < ./INPDIR/h2o.in.03.${INPUT} > ./OUTDIR.$INPUT/h2o.out.03.${INPUT}"
done

In [12]:
print(time.time() - stime)
stime = time.time()

188.18521285057068
